# Download a data file from the UCL server

We're using a small [example data set](http://data.cortexlab.net/singlePhase3/) recorded by [Nick Steinmetz](http://www.nicksteinmetz.com/) at UCL.

In [ ]:
import urllib.request
file_url = 'http://data.cortexlab.net/singlePhase3/data/rawDataSample.bin'
file_name = "rawDataSample.bin"

urllib.request.urlretrieve(file_url, file_name)

# Use of the `HSDetection` class

## Important note

This is set up to run in the local directory structure.
To have a working HS2, run

``python setup.py build_ext --inplace``

in the project root directory.

In [ ]:
import sys
sys.path.append('../')
from herdingspikes.hs2 import HSDetection
from herdingspikes.probe import NeuroPixel
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# raw data file (as downloaded above)
data_path = "rawDataSample.bin"

# detection parameters
to_localize = True
cutout_start = 6
cutout_end = 30
num_com_centers = 2
threshold = 20
file_directory = 'results/'
file_name = 'ProcessedSpikesNeuropixel'

In [ ]:
Probe = NeuroPixel(data_path, neighbor_radius=120, masked_channels=[36, 75, 112, 151, 188, 227, 264, 303, 340, 379, 384])
H = HSDetection(Probe, to_localize, num_com_centers, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name, 
                file_directory_name=file_directory, decay_filtering=True, save_all=True)

In [ ]:
Probe.show()

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()


In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(15, window_size=100)

In [ ]:
plt.figure(figsize=(14, 3))
ax = plt.subplot(111)
H.PlotAll(invert=True, s=1, alpha=0.2, ax=ax)
ax.set_xlim((-100,600))

# Use of the `HSClustering` class

In [ ]:
from herdingspikes.hs2 import HSClustering

# Load from file
#C = Clustering(['results/ProcessedSpikes_mea1k.bin', 'results/ProcessedSpikes_mea1k.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
%%time
# Compute features
C.ShapePCA(pca_ncomponents=2, pca_whiten=True);

In [ ]:
%%time
# Cluster all spikes

# This shows how a (potentially relatively small) subset of spikes 
# can be used to form clusters. All spikes are then assigned
# to these clusters in batches. This allows clustering of very large 
# recordings (also multiple files) without exhausting memory.

C.CombinedClustering(alpha=7, bandwidth=5.0, bin_seeding=False, n_jobs=-1, cluster_subset=100000)

In [ ]:
plt.figure(figsize=(14, 4))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, ax=ax, max_show=100000, show_labels=False)
plt.title("MeanShift, bandwidth=5, no bin seeding");
ax.set_xlim((30,330))


In [ ]:
units = range(16)
C.PlotShapes(units)

In [ ]:
C.PlotNeighbourhood(2, radius=6, alpha=0.8);

In [ ]:
# Save the results

C.SaveHDF5(file_name+"_sorted.hdf5")